In [ ]:
!git clone https://github.com/datvodinh10/project-DD.git

In [ ]:
config = {
    'transformer':{
        'vocab_size':1000,
        'embed_size': 256,
        'num_heads':4,
        'num_layers':2,
        'max_len': 10_000,
        'dropout':0.2,
        'bias':False,
        'device':torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
        'lr':1e-4,
        'batch_size':32,
        'block_size':256,
        'n_iter':5,
    }
}

In [17]:
import torch
import torch.nn.functional as F

def pad_sequence(X: torch.Tensor, max_len: int):
    # Determine the current length of the sequence
    seq_len = X.size(0)

    # Compute the required padding length
    pad_len = max_len - seq_len

    # Pad the sequence tensor with zeros
    padded_X = torch.cat([X,torch.zeros((pad_len,*X.shape[1:]))],dim=0)

    return padded_X

In [18]:
pad_sequence(torch.tensor([1,2]),5)

tensor([1., 2., 0., 0., 0.])

In [19]:
X = torch.tensor([1,2])
pad_len = 3
torch.zeros((pad_len,*X.shape[1:]))

tensor([0., 0., 0.])